## 실습 [26-2]  
### 1. 실습명 : GPT-2로 문장 생성하기
### 2. 실습 목적 및 설명
* GPT-2-Simple 코드를 이용해 GPT-2 Medium 모델을 파인튜닝 해본다.
* nltk에서 제공하는 영화 리뷰 데이터셋을 이용하여 문장을 생성한다.
* 입력 데이터로는 어떠한 전처리도 거치지 않은 원본 텍스트를 사용한다.

### 3. 관련 장(챕터) : 26.2.1 OpenGPT-2를 이용한 문장 생성
### 4. 코드

In [0]:
"""
본 실습에서는 GPT-2 Medium 모델(355M)로 문장 생성을 진행한다. 
GPT-2 모델은 전체 파라미터 훈련에 상당한 시간이 소요되지만 Colab 환경 특성상 런타임 제약이 존재하므로 
간단히 사전 학습된 모델을 불러와 특정 데이터셋으로 파인튜닝(fine-tuning) 후 문장을 생성하도록 한다. 
코드는 MIT의 Max Woolf(@minimaxir)가 배포한 gpt-2-simple 모델을 사용한다. 
gpt-2-simple 모델을 통해 OpenAI에서 배포한 GPT-2모델을 간편하게 사용할 수 있다. 
gpt-2-simple의 원본 소스 코드는‘https://github.com/minimaxir/gpt-2-simple'에서 확인할 수 있다.
"""
# Colab 환경에서 실행할 경우 빠른 모델 훈련을 위해 런타임 유형을 GPU로 설정하는 것이 좋다.
# gpt-2-simple의 경우 현재 텐서플로우2.0을 지원하지 않으므로 tensorflow_version 1.x로 설정한다. 
%tensorflow_version 1.x
import tensorflow as tf
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import nltk
nltk.download('punkt')
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
import gpt_2_simple as gpt2
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [0]:
# 데이터셋을 가져와 내용을 확인한다.
raw_data = movie_reviews.raw()
print(raw_data[:150])

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continue


In [0]:
ls /content/

checkpoint/  models/  move_review_file.txt  movie_review_file.txt  sample_data/


In [0]:
# 데이터셋을 txt 파일로 만들어 현재 위치(/content/)에 저장한다.
data_file = "movie_review_file.txt"
with open(data_file, 'w') as f:
  f.write(raw_data)

In [0]:
tf.reset_default_graph()

In [0]:
# GPT-2 모델을 다운로드 받은 후 데이터셋 파일을 이용해 파인 튜닝을 진행한다.
model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
  print(f"Downloading {model_name} model...")
  gpt2.download_gpt2(model_name = model_name)

sess = gpt2.start_tf_sess()

# 파라미터는 필요에 맞게 설정한다. 
# 본 실습에서는 빠른 진행을 위해 스텝(steps) 값을 100으로 설정하였다.
gpt2.finetune(sess, data_file, model_name = model_name, 
              run_name = 'run1', print_every=1, sample_every=10, 
              save_every=50, learning_rate=0.0001, sample_length=500, 
              optimizer='adam', steps=100)

Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.07s/it]


dataset has 1844986 tokens
Training...
[1 | 100.67] loss=3.00 avg=3.00
[2 | 187.04] loss=3.27 avg=3.14
[3 | 273.48] loss=3.30 avg=3.19
[4 | 359.97] loss=3.22 avg=3.20
[5 | 446.34] loss=3.55 avg=3.27
[6 | 532.80] loss=3.16 avg=3.25
[7 | 619.26] loss=3.61 avg=3.30
[8 | 705.57] loss=3.35 avg=3.31
[9 | 792.23] loss=3.05 avg=3.28
[10 | 878.92] loss=3.11 avg=3.26
======== SAMPLE 1 ========
 beneath a layer of mist. His skin suddenly became a shade of gold, his hair the color of a rose, his beard the color of roses of all varieties.

The moment the sun rose, the sun rose with him. I could see with my own eyes that he would never leave his place in this world. He would be nothing more than a human skeleton.

I did not even notice the sun until after the sun had set.

If I was to say that he would die right here under the tree, then he would. Although his death would not be an instant or an explosion, it would be a terrible loss. In comparison to the many casualties he would suffer, that would 

In [0]:
# 훈련 모델을 저장하는 경우
# 모델을 gdrive에 마운트한다.
from google.colab import auth
auth.authenticate_user()
gpt2.mount_gdrive()

In [0]:
# 체크포인트를 gdrive에 복사한다.
gpt2.copy_checkpoint_to_gdrive()

In [0]:
# 테스트(문장 생성)를 위해 글로벌 변수를 초기화한 후 문장을 생성 한다.
# 훈련시 사용한 run_name을 명시하고 문장을 시작할 문자열을 정한다.
init = tf.global_variables_initializer()
sess.run(init)
gpt2.generate(sess, run_name='run1', prefix= 'this movie')

<참고>


> https://github.com/minimaxir/gpt-2-simple

